## InstructOR - A multitask custom embedding model for task based applications, made easier with LanceDB
![instruct](embeddings11.png)

### Installing all dependencies

In [51]:
!pip install git+https://github.com/lancedb/lancedb.git@main#subdirectory=python

  Cloning https://github.com/lancedb/lancedb.git (to revision main) to c:\users\lenovo\appdata\local\temp\pip-req-build-yk610zm1
  Resolved https://github.com/lancedb/lancedb.git to commit 479f471c145abc9818a85ceee32b7e364fd4abd2
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/lancedb/lancedb.git 'C:\Users\Lenovo\AppData\Local\Temp\pip-req-build-yk610zm1'


In [5]:
!pip install InstructorEmbedding sentence-transformers torch pandas

If you want to calculate customized embeddings for specific sentences, you may follow the unified template to write instructions:

"Represent the [**domain**] [**text_type**] for [**task_objective**]:"

Here are some examples:

- "Represent the **Science** **sentence**:"
- "Represent the **Financial** **statement**:"
- "Represent the **Wikipedia** **document** for **retrieval**:"
- "Represent the **Wikipedia** **question** for **retrieving supporting documents**:"

### Importing neccessary functionalities

In [1]:
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry
from lancedb.embeddings import InstructorEmbeddingFunction

### Calling the embedding model from LanceDB embedding's API

In [2]:
instructor = get_registry().get("instructor").create(
                            source_instruction="represent the document for retreival",
                            query_instruction="represent the document for most similar definition"
                            )

class Schema(LanceModel):
    vector: Vector(instructor.ndims()) = instructor.VectorField()
    text: str = instructor.SourceField()

# Creating LanceDB table
db = lancedb.connect("~/.lancedb")
tbl = db.create_table("intruct-multitask", schema=Schema, mode="overwrite")

C:\Users\Lenovo\Documents\instruct\lenv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


### Adding Data to the Table

In [3]:
data_f1= [
    {"text": "Aspirin is a widely-used over-the-counter medication known for its anti-inflammatory and analgesic properties. It is commonly used to relieve pain, reduce fever, and alleviate minor aches and pains."},
    {"text": "Amoxicillin is an antibiotic medication commonly prescribed to treat various bacterial infections, such as respiratory, ear, throat, and urinary tract infections. It belongs to the penicillin class of antibiotics and works by inhibiting bacterial cell wall synthesis."},
    {"text": "Atorvastatin is a lipid-lowering medication used to manage high cholesterol levels and reduce the risk of cardiovascular events. It belongs to the statin class of drugs and works by inhibiting an enzyme involved in cholesterol production in the liver."},
    {"text": "The Theory of Relativity is a fundamental physics theory developed by Albert Einstein, consisting of the special theory of relativity and the general theory of relativity. It revolutionized our understanding of space, time, and gravity."},
    {"text": "Photosynthesis is a vital biological process by which green plants, algae, and some bacteria convert light energy into chemical energy in the form of glucose, using carbon dioxide and water."},
    {"text": "The Big Bang Theory is the prevailing cosmological model that describes the origin of the universe. It suggests that the universe began as a singularity and has been expanding for billions of years."},
    {"text": "Compound Interest is the addition of interest to the principal sum of a loan or investment, resulting in the interest on interest effect over time."},
    {"text": "Stock Market is a financial marketplace where buyers and sellers trade ownership in companies, typically in the form of stocks or shares."},
    {"text": "Inflation is the rate at which the general level of prices for goods and services is rising and subsequently purchasing power is falling."},
    {"text": "Diversification is an investment strategy that involves spreading your investments across different asset classes to reduce risk."},
    {"text": "Liquidity refers to how easily an asset can be converted into cash without a significant loss of value. It's a key consideration in financial management."},
    {"text": "401(k) is a retirement savings plan offered by employers, allowing employees to save and invest a portion of their paycheck before taxes."},
    {"text": "Ballet is a classical dance form that originated in the Italian Renaissance courts of the 15th century and later developed into a highly technical art."},
    {"text": "Rock and Roll is a genre of popular music that originated and evolved in the United States during the late 1940s and early 1950s, characterized by a strong rhythm and amplified instruments."},
    {"text": "Cuisine is a style or method of cooking, especially as characteristic of a particular country, region, or establishment."},
    {"text": "Renaissance was a cultural, artistic, and intellectual movement that"},
    {"text": "Neutrino is subatomic particles with very little mass and no electric charge. They are produced in various nuclear reactions, including those in the Sun, and play a significant role in astrophysics and particle physics."},
    {"text": "Higgs Boson is a subatomic particle that gives mass to other elementary particles. Its discovery was a significant achievement in particle physics."},
    {"text": "Quantum Entanglement is a quantum physics phenomenon where two or more particles become connected in such a way that the state of one particle is dependent on the state of the other(s), even when they are separated by large distances."},
    {"text": "Genome Sequencing is the process of determining the complete DNA sequence of an organism's genome. It has numerous applications in genetics, biology, and medicine."},
]

tbl.add(data_f1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## First use case - Semantic Search with LanceDB

In [4]:
#LanceDB supports full text search, so there is no need of embedding the Query
query = "amoxicillin"
result = tbl.search(query).limit(1).to_pandas()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
#printing the output
print(result)

                                              vector  \
0  [-0.024510665, 0.0005563133, 0.0288403, 0.0807...   

                                                text  _distance  
0  Amoxicillin is an antibiotic medication common...   0.163671  


## Same Input Data with Different Instruction Pair

In [6]:
instructor = get_registry().get("instructor").create(
                            source_instruction="represent the captions",
                            query_instruction="represent the captions for retrieving duplicate captions"
                            )

class Schema(LanceModel):
    vector: Vector(instructor.ndims()) = instructor.VectorField()
    text: str = instructor.SourceField()

db = lancedb.connect("~/.lancedb")
tbl = db.create_table("intruct-multitask", schema=Schema, mode="overwrite")

load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
data_f2 = [
    {"text": "Aspirin is a widely-used over-the-counter medication known for its anti-inflammatory and analgesic properties. It is commonly used to relieve pain, reduce fever, and alleviate minor aches and pains."},
    {"text": "Amoxicillin is an antibiotic medication commonly prescribed to treat various bacterial infections, such as respiratory, ear, throat, and urinary tract infections. It belongs to the penicillin class of antibiotics and works by inhibiting bacterial cell wall synthesis."},
    {"text": "Atorvastatin is a lipid-lowering medication used to manage high cholesterol levels and reduce the risk of cardiovascular events. It belongs to the statin class of drugs and works by inhibiting an enzyme involved in cholesterol production in the liver."},
    {"text": "The Theory of Relativity is a fundamental physics theory developed by Albert Einstein, consisting of the special theory of relativity and the general theory of relativity. It revolutionized our understanding of space, time, and gravity."},
    {"text": "Photosynthesis is a vital biological process by which green plants, algae, and some bacteria convert light energy into chemical energy in the form of glucose, using carbon dioxide and water."},
    {"text": "The Big Bang Theory is the prevailing cosmological model that describes the origin of the universe. It suggests that the universe began as a singularity and has been expanding for billions of years."},
    {"text": "Compound Interest is the addition of interest to the principal sum of a loan or investment, resulting in the interest on interest effect over time."},
    {"text": "Stock Market is a financial marketplace where buyers and sellers trade ownership in companies, typically in the form of stocks or shares."},
    {"text": "Inflation is the rate at which the general level of prices for goods and services is rising and subsequently purchasing power is falling."},
    {"text": "Diversification is an investment strategy that involves spreading your investments across different asset classes to reduce risk."},
    {"text": "Liquidity refers to how easily an asset can be converted into cash without a significant loss of value. It's a key consideration in financial management."},
    {"text": "401(k) is a retirement savings plan offered by employers, allowing employees to save and invest a portion of their paycheck before taxes."},
    {"text": "Ballet is a classical dance form that originated in the Italian Renaissance courts of the 15th century and later developed into a highly technical art."},
    {"text": "Rock and Roll is a genre of popular music that originated and evolved in the United States during the late 1940s and early 1950s, characterized by a strong rhythm and amplified instruments."},
    {"text": "Cuisine is a style or method of cooking, especially as characteristic of a particular country, region, or establishment."},
    {"text": "Renaissance was a cultural, artistic, and intellectual movement that"},
    {"text": "Neutrino is subatomic particles with very little mass and no electric charge. They are produced in various nuclear reactions, including those in the Sun, and play a significant role in astrophysics and particle physics."},
    {"text": "Higgs Boson is a subatomic particle that gives mass to other elementary particles. Its discovery was a significant achievement in particle physics."},
    {"text": "Quantum Entanglement is a quantum physics phenomenon where two or more particles become connected in such a way that the state of one particle is dependent on the state of the other(s), even when they are separated by large distances."},
    {"text": "Genome Sequencing is the process of determining the complete DNA sequence of an organism's genome. It has numerous applications in genetics, biology, and medicine."},
]

tbl.add(data_f2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
#same query, but for the differently embed data
query = "amoxicillin"
result = tbl.search(query).limit(1).to_pandas()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
#showing the result
print(result)

                                              vector  \
0  [-0.024483299, 0.000932854, 0.033273745, 0.077...   

                                                text  _distance  
0  Amoxicillin is an antibiotic medication common...    0.18135  


### We can see that the **_distance** value for different instructions are different, this clearly indicates that the instructions have some effect on the embedding

## Second use case - Question Answering with Pinecone

### Calling embedding model with different instruction pair

In [10]:
instructor = get_registry().get("instructor").create(
                            source_instruction="represent the docuement for retreival",
                            query_instruction="represent the query for retrieval"
                            )

class Schema(LanceModel):
    vector: Vector(instructor.ndims()) = instructor.VectorField()
    text: str = instructor.SourceField()

db = lancedb.connect("~/.lancedb-qa")
tbl = db.create_table("intruct-multitask-qa", schema=Schema, mode="overwrite")

load INSTRUCTOR_Transformer
max_seq_length  512


In [11]:
data_qa = [ { "text": "A canvas painting is artwork created on a canvas surface using various painting techniques and mediums like oil, acrylic, or watercolor. It is popular in traditional and contemporary art, displayed in galleries, museums, and homes."},
    { "text": "A cinema, also known as a movie theater or movie house, is a venue where films are shown to an audience for entertainment. It typically consists of a large screen, seating arrangements, and audio-visual equipment to project and play movies."},
    { "text": "A pocket watch is a small, portable timekeeping device with a clock face and hands, designed to be carried in a pocket or attached to a chain. It is typically made of materials such as metal, gold, or silver and was popular during the 18th and 19th centuries."},
    { "text": "A laptop is a compact and portable computer with a keyboard and screen, ideal for various tasks on the go. It offers versatility for browsing, word processing, multimedia, gaming, and professional work."}
]

tbl.add(data_qa)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
query = "what is a cinema"
result = tbl.search(query).limit(1).to_pandas()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
print(result)

                                              vector  \
0  [0.021844529, 0.0017777127, 0.022723941, 0.049...   

                                                text  _distance  
0  A cinema, also known as a movie theater or mov...   0.131036  


Thanks, for more such examples, please visit [LanceDB](https://github.com/lancedb/vectordb-recipes/tree/main)